# Environment monitor

This notebook is the environment monitor. It will log all readings to Adafruit IO and show readings on the LCD screen.

In [91]:
import time
import ST7735
from ltr559 import LTR559
from bme280 import BME280
from subprocess import PIPE, Popen
from PIL import Image, ImageDraw, ImageFont
from fonts.ttf import RobotoMedium as UserFont
from IPython.display import clear_output, display, update_display

In [92]:
import json
from Adafruit_IO import Client, Feed, RequestError

In [93]:
ADAFRUIT_IO_USERNAME = "tproffen"
ADAFRUIT_IO_KEY = "ed44b8031ec34d1fb2eda6f4d8a0c832"

In [94]:
# Create an instance of the REST client.
aio = Client(ADAFRUIT_IO_USERNAME, ADAFRUIT_IO_KEY)

In [95]:
# BME280 temperature/pressure/humidity sensor
bme280 = BME280()

In [96]:
# Proximity/Light sensor
ltr559 = LTR559()

#### Some useful functions

In [97]:
def getFeed(name):
    try:
        feedID = aio.feeds(name)
    except RequestError:
        feed = Feed(name=name)
        feedID = aio.create_feed(feed)
    return feedID

## Main loop

This is the main loop reading values and sending them to the cloud and updating the LCD display. Press Control-C to aboort of click the stop button.

In [ ]:
# Get feeds
tempFeed = getFeed("temperature")
humidFeed = getFeed("humidity")
pressFeed = getFeed("pressure")
lightFeed = getFeed("brightness")

# Time between readings
delay = 20

# The main loop
while True:
    # Read pressure
    p = bme280.get_pressure()
    t = bme280.get_temperature()
    h = bme280.get_humidity()
    l = ltr559.get_lux()
    
    out="T: {0:.1f} C - Humidity: {1:.1f} % - Pressure: {2:.0f} hPa".format(t,h,p)
    clear_output()
    print(out)
    
    aio.send_data(tempFeed.key, t)
    aio.send_data(humidFeed.key, h)
    aio.send_data(pressFeed.key, p)
    aio.send_data(lightFeed.key, l)
    
    time.sleep(delay)


T: 27.0 C - Humidity: 34.9 % - Pressure: 988 hPa
